## 日本語 Tokenizer

pip install SudachiPy sudachidict_core # or sudachidict_full


In [1]:
from pprint import pprint
import re, glob, time
import numpy as np
import pandas as pd
from sudachipy import Dictionary, SplitMode

In [2]:
# pip install janome
from janome.tokenizer import Tokenizer

# Janomeのインスタンス化
tokenizer = Tokenizer()

text = '今日は、良い天気です。'
tokens = tokenizer.tokenize(text)
pprint([(t.surface, t.part_of_speech) for t in tokens])

[('今日', '名詞,副詞可能,*,*'),
 ('は', '助詞,係助詞,*,*'),
 ('、', '記号,読点,*,*'),
 ('良い', '形容詞,自立,*,*'),
 ('天気', '名詞,一般,*,*'),
 ('です', '助動詞,*,*,*'),
 ('。', '記号,句点,*,*')]


In [4]:
from sudachipy import Dictionary, SplitMode
tokenizer = Dictionary(dict='full').create()

from sudachipy import dictionary
print([dictionary.Dictionary().lookup('撹拌条件')]) # User_dic確認
print([dictionary.Dictionary().lookup('HbA1c')]) # User_dic確認
print([dictionary.Dictionary().lookup('hba1c')]) # User_dic確認

def is_hiragana(text):
    return bool(re.fullmatch(r'[ぁ-ゔ]*', text))
    
text = '今日は、良い天気です。'
pprint([(t.surface(), t.part_of_speech()) for t in tokenizer.tokenize(text)])

text = '国会議事堂前駅'
tokens = tokenizer.tokenize(text) # Defult = SplitMode.C
pprint([(t.surface(), t.part_of_speech()) for t in tokens])
pprint([(t.surface(), t.part_of_speech()) for t in tokenizer.tokenize(text, SplitMode.A)]) # 超細かい
pprint([(t.surface(), t.part_of_speech()) for t in tokenizer.tokenize(text, SplitMode.B)]) # 細かい
pprint([(t.surface(), t.part_of_speech()) for t in tokenizer.tokenize(text, SplitMode.C)])

[<MorphemeList[
  <Morpheme(撹拌条件, 0:4, (1, 0))>,
]>]
[<MorphemeList[
]>]
[<MorphemeList[
  <Morpheme(hba1c, 0:5, (1, 44))>,
]>]
[('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
[('国会議事堂前駅', ('名詞', '固有名詞', '一般', '*', '*', '*'))]
[('国会', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('議事', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('堂', ('接尾辞', '名詞的', '一般', '*', '*', '*')),
 ('前', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('駅', ('名詞', '普通名詞', '一般', '*', '*', '*'))]
[('国会', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('議事堂', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('前', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('駅', ('名詞', '普通名詞', '一般', '*', '*', '*'))]
[('国会議事堂前駅', ('名詞', '固有名詞', '一般', '*', '*', '*'))]


### Note Data

In [2]:
def read_dataset(doc_dir='notes', trace=False): # -> dic
    """実験ノート(md)の読み込み"""
    indata_section_names = ['目的','材料','実験手順']

    # doc_dir以下のすべての .md ファイルを取得
    md_files = glob.glob(f"{doc_dir}/*.md")
    if trace:
        print(len(md_files))

    # レシピを辞書に格納
    notes_dic = {}
    for i, file_path in enumerate(md_files, start=1):
        with open(file_path, "r", encoding="utf-8") as f:
            note = f.read().strip()
            name = re.search(r"^# (.+)$", note, re.MULTILINE).group(1).strip()
            notes_dic[name] = note.strip()
    print(f"\ndoc_dir={doc_dir}, 読みこんだレシピの数: {len(notes_dic)}")
    if trace:
        first_key = list(notes_dic.keys())[0]; print(notes_dic[first_key]) # 最初のレシピ

    # セクションごとに辞書化
    section_pattern = re.compile(r'^## (.*?)\n(.*?)(?=\n## |\Z)', re.MULTILINE | re.DOTALL)
    for i, (name, note) in enumerate(notes_dic.items()):
        if trace:
            print(f"## {name} ====")
        matches = section_pattern.findall(note)  # セクションごとに分離辞書化
        result_dic = {title.strip(): content.strip() for title, content in matches}

        # Section名を標準化
        for (std_name, indata_name) in zip(SECTION_NAMES, indata_section_names):
            if std_name != indata_name:
                result_dic[std_name] = result_dic.pop(indata_name)

        # allの追加
        result_dic['all'] = '\n\n'.join([f"## {s}\n\n{result_dic[s]}" for s in SECTION_NAMES])

        notes_dic[name] = result_dic
    first_key = list(notes_dic.keys())[0]
    last_key  = list(notes_dic.keys())[-1]
    print(f"\n辞書化したノートの数: {len(notes_dic)}" +\
          f"\nFirst: {first_key} ~ Last: {last_key} \n{notes_dic[first_key].keys()}")
    if trace:
        pprint(notes_dic[first_key])

    return notes_dic

SECTION_NAMES = ['目的','材料','手順'] # 正規化時に付与するセクション名
doc_dir = '実験ノート1206'
notes_dic = read_dataset(doc_dir, trace=False)   # データの読み込み


doc_dir=実験ノート1206, 読みこんだレシピの数: 340

辞書化したノートの数: 340
First: 実験ノート ID1-1 ~ Last: 実験ノート ID4-9 
dict_keys(['目的', '材料', '手順', 'all'])


### tokenize_and_filter_by_pos

In [3]:
### すだち
from sudachipy import Dictionary, SplitMode
tokenizer = Dictionary(dict='full').create()

def tokenize(text, splitmode='C', pos=False):

    match splitmode.upper():
        case 'A':
            splitmode = SplitMode.A # 超細かい
        case 'B':
            splitmode = SplitMode.B # 細かい
        case _:
            splitmode = SplitMode.C

    tokens = tokenizer.tokenize(text, splitmode) # Defult = SplitMode.C
    if pos:
        pprint([(t.surface(), t.part_of_speech()) for t in tokens])
    tokens = [t.surface() for t in tokens]
    return tokens

def tokenize_and_filter_by_pos(
    text, 
    target_pos=['名詞','動詞','形容詞','副詞','接頭詞'],
    pos=False
):
    # 入力文から指定された品詞のトークンを抽出
    tokens = list(tokenizer.tokenize(text))
    if pos:
        pprint([(t.surface(), t.part_of_speech()) for t in tokens])
    parts_of_speech = [t.part_of_speech() for t in tokens]                             # 品詞情報を抽出
    is_target_pos = [bool(set(p) & set(target_pos)) and 
                     not (bool(set(p) & set(['動詞'])) and
                          bool(set(p) & set(['非自立', '非自立可能'])))
                                                             for p in parts_of_speech] # 必要品詞で抽出
    #pprint(is_target_pos)
    extracted_tokens = [t.surface() for (t, f) in zip(tokens, is_target_pos) if f]
    return extracted_tokens


pprint(tokenize('① 今日は、良い天気です。', pos=True))
pprint(tokenize_and_filter_by_pos('① 今日は、良い天気です。', pos=True))

[('①', ('名詞', '数詞', '*', '*', '*', '*')),
 (' ', ('空白', '*', '*', '*', '*', '*')),
 ('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
['①', ' ', '今日', 'は', '、', '良い', '天気', 'です', '。']
[('①', ('名詞', '数詞', '*', '*', '*', '*')),
 (' ', ('空白', '*', '*', '*', '*', '*')),
 ('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
['①', '今日', '良い', '天気']


In [4]:
ZEN2HAN = str.maketrans(
    "".join(chr(0xff01 + i) for i in range(94))+"　", # Zen
    "".join(chr(0x21   + i) for i in range(94))+" "   # Han
)

#分解パターンの定義
# 正規表現
# 末尾の数量、単位の分離 (省略可)
qty_unit_pattern = re.compile(
    r"""
    \s*[:：]*\s*             # 0個以上の空白、セパレータ、空白
    (?P<qty>[\d.,]+)         # 数量 (数値)
    \s*                      # 0個以上の空白
    (?P<unit>[kKmnμµ]{0,1}[a-zA-Z]+|[日時分秒℃度%]|°[CK]|rpm|mol|min|sec)?  # 単位 (省略可) <= 工夫の余地あり
    $                        # 末尾
    """,
    re.VERBOSE
)

def parse_material(line, trace=False):
    '''
    材料行を分解
    材料と数量単位は、空白またはセパレータ[:：]で分かち書きされている
    以下に分解
    1. 先頭[-*・] (無い場合がある) => 無視
    2. 項番: 例: ①,... (1),...   (無い場合がある) => 無視
    3. 材料: 材料名, 補足
      3.1 材料名 数値を含む場合あり => 変数 mate に格納
      3.2 補足「()（）」で囲まれた補足情報が続く場合もあり => ()含め変数 add に格納
    4. 数量, 単位: 無い場合もあり、 数量と単位に間に空白がある場合と無い場合がある
    　4.1 数量(数値、半角または全角、「,.」を含む) => 変数 qtyに格納　
      4.2 単位((L,g)x(そのまま、k,m,μ))、ignore case) => 変数 unit に格納
    '''
    line = line.strip().replace("　", " ")                 # 前後余白を削除、全角スペースを半角に
    line = re.sub(r'^[\-\*]+\s*', '', line)                # 先頭の "-", "*", "•" を除去
    line = re.sub(r'^\s*(\(?\d+\)?|[①-⑳])\s*', '', line)   # 項番（①, ②, (1), 1. など）を除去
    line = line.translate(ZEN2HAN).strip()                 # 数字記号は半角に
    if trace:
        print(f"## {line}")

    # 1. 正規化：セパレータは「空白 or : or ：」
    #    最後の数量部分を優先的に捉えるため "数量+単位" をパターン化して先に抽出
    qty = ''; unit = ''
    m = qty_unit_pattern.search(line)
    if m:
        qty  = m.group('qty')
        unit = m.group('unit')
        # 数量部分を取り除いて「材料側」だけにする
        line_material = line[:m.start()].rstrip()
    else:
        # 数量・単位なし
        line_material = line.strip()

    # 2. 材料名末尾に続く補足括弧を抽出（複数対応）
    #   材料名内部にある括弧は mate に含める
    add = ''
    add_list = []

    # 補足括弧は、文末方向に連続しているものだけ抽出
    add_pattern = re.compile(r"(\([^)]*\)|（[^）]*）)$")

    tmp = line_material
    while True:
        m = add_pattern.search(tmp)
        if not m:
            break
        add_list.insert(0, m.group(1))  # 先頭に挿入して順序維持
        tmp = tmp[:m.start()].rstrip()

    if add_list:
        add = ''.join(add_list)
        mate = tmp.strip()
    else:
        mate = line_material.strip()

    mate_dic = {'material': mate, 'add': add, 'qty': qty, 'unit': unit}
    if trace:
        print(f"## ## {mate_dic}")
    mate_dic = {k: v for k, v in mate_dic.items() if v != '' and v is not None} # 値が''の要素をdrop
    if trace:
        print(f"## ## {mate_dic}")

    return mate_dic

In [5]:
text = notes_dic['実験ノート ID1-2']['材料'].translate(ZEN2HAN).strip()
text1 = notes_dic['実験ノート ID3-30']['材料'].translate(ZEN2HAN).strip()
text2 = notes_dic['実験ノート ID3-3']['材料'].translate(ZEN2HAN).strip()
text3 = notes_dic['実験ノート ID3-20']['材料'].translate(ZEN2HAN).strip()
text4 = notes_dic['実験ノート ID3-50']['材料'].translate(ZEN2HAN).strip()
text = text1+'\n\n\n'+text2+'\n\n\n'+text3+'\n\n\n'+text4
lines = list(set(text.split('\n')))
print(lines)

['', '- ⑩ グリセロール:3 g', '- ⑨ マルチトール:4 g', '- ⑧ 純水:15 g', '- ⑨ マルチトール:3.6 g', '- ② HEPES:2.2 g', '- ⑥ 純水:23 g', '- ⑦ BSA:1 g', '- ⑧ 純水:13 g', '- ③ IgG(粒径 120 nm):1.4 g', '- ⑥ 純水:24 g', '- ⑤ HbA1c反応抗体(マウスモノクローナル,全長IgG,可溶性):10 g', '- ① 純水:27 g', '- ④ IgG(粒径 156.06 nm):1 g', '- ⑩ ソルビトール:1 g', '- ⑦ カゼイン:1 g', '- ⑨ グリセロール:4 g', '- ⑦ BSA:0.95 g', '- ⑦ カゼイン:1.2 g', '- ④ IgG(粒径 150 nm):1 g', '- ③ IgG(粒径 100 nm):1 g', '- ① 純水:28 g', '- ② Glycine:2 g', '- ② HEPES:1.8 g', '- ① 純水:30 g', '- ④ IgG(粒径 200 nm):1 g', '- ⑥ 純水:25 g', '- ④ IgG(粒径 170 nm):1.4 g', '- ③ IgG(粒径 102 nm):1 g', '- ⑩ マルチトール:2 g', '- ③ IgG(粒径 110 nm):1 g', '- ② NaOH:1.5 g']


In [6]:
text = []
for i in range(100):
    for line in notes_dic[f"実験ノート ID1-{i+1}"]['材料'].translate(ZEN2HAN).strip().split('\n'):
        mate = parse_material(line)['material']
        text.append(mate)
for i in range(90):
    for line in notes_dic[f"実験ノート ID2-{i+1}"]['材料'].translate(ZEN2HAN).strip().split('\n'):
        mate = parse_material(line)['material']
        text.append(mate)
for i in range(100):
    for line in notes_dic[f"実験ノート ID3-{i+1}"]['材料'].translate(ZEN2HAN).strip().split('\n'):
        mate = parse_material(line)['material']
        text.append(mate)
for i in range(50):
    for line in notes_dic[f"実験ノート ID4-{i+1}"]['材料'].translate(ZEN2HAN).strip().split('\n'):
        mate = parse_material(line)['material']
        text.append(mate)
text = '\n'.join(list(set(text)))
print(text)

原料2:C8–C10ミックス脂肪酸
MOPS
原料2:C7–C9ミックス脂肪酸 4.4 mol
原料1:ネオペンチルグリコール
HbA1c捕捉抗体1
カゼイン
ソルビトール
原料2:ラウリン酸
原料3:スルホン化樹脂系固体酸触媒 1.0 wt%
原料1:ドデカノール
原料3:p-トルエンスルホン酸 0.25 mol%
Tris
原料5:フェノール系酸化防止剤 0.01 mol
原料4:活性白土 0.05 mol 相当
KCl
原料2:イソステアリン酸
HbA1c検出抗体1
原料3:p-トルエンスルホン酸 0.4 mol%
EPPS
Bis-Tris
原料1:グリセリン
HEPES
HbA1c検出抗体2
CHES
精製水
原料4:トルエン 反応系に対して約30 wt%
原料4:C8–C10ミックス脂肪酸
グリセロール
原料1:トリメチロールプロパン
原料3:p-トルエンスルホン酸 0.8 mol%
BSA
NaOH
HbA1c捕捉抗体B
原料2:ネオペンチルグリコール
HbA1c捕捉抗体2
原料3:ノナン酸
原料3:スルホン化樹脂系固体酸触媒
原料4:アミン系酸化防止剤 0.01 mol
原料3:p-トルエンスルホン酸 0.5 mol%
ゼラチン
原料5:p-トルエンスルホン酸 0.5 mol%
原料4:p-トルエンスルホン酸 0.5 mol%
IgGコートラテックス
原料3:ペンタエリスリトール
HbA1c検出抗体A
HbA1c検出抗体B
Glycine
原料1:2-エチルヘキサノール
原料2:C7–C9ミックス脂肪酸
原料4:ノナン酸
原料5:C7–C9ミックス脂肪酸
純水
NaHCO3
原料2:C7–C9ミックス脂肪酸 2.2 mol
原料4:フェノール系酸化防止剤 0.01 mol
HbA1c反応抗体
原料2:ノナン酸
NaCl
原料3:C8–C10ミックス脂肪酸
PBS
マルチトール
カイゼン
原料2:ペンタエリスリトール
原料2:高純度C8–C10ミックス脂肪酸
IgG
原料4:トルエン 2.0 mol
f(ab')2
原料3:C7–C9ミックス脂肪酸 2.2 mol
原料1:ペンタエリスリトール
HbA1c捕捉抗体A
原料3:C7–C9ミックス脂肪酸
f(ab')2コートラテックス
原料2:リシノール酸
原料3:p-トルエンスルホン酸 0.3 mol%


In [7]:
print(tokenize(text))
print(tokenize_and_filter_by_pos(text, pos=True))

['原料', '2', ':', 'C', '8', '–', 'C', '10', 'ミックス', '脂肪酸', '\n', 'MOPS', '\n', '原料', '2', ':', 'C', '7', '–', 'C', '9', 'ミックス', '脂肪酸', ' ', '4.4', ' ', 'mol', '\n', '原料', '1', ':', 'ネオペンチルグリコール', '\n', 'HbA1c', '捕捉抗体', '1', '\n', 'カゼイン', '\n', 'ソルビトール', '\n', '原料', '2', ':', 'ラウリン酸', '\n', '原料', '3', ':', 'スルホン化', '樹脂', '系', '固体', '酸', '触媒', ' ', '1.0', ' ', 'wt', '%', '\n', '原料', '1', ':', 'ドデカノール', '\n', '原料', '3', ':', 'p-トルエンスルホン酸', ' ', '0.25', ' ', 'mol', '%', '\n', 'Tris', '\n', '原料', '5', ':', 'フェノール系', '酸化防止剤', ' ', '0.01', ' ', 'mol', '\n', '原料', '4', ':', '活性白土', ' ', '0.05', ' ', 'mol', ' ', '相当', '\n', 'KCl', '\n', '原料', '2', ':', 'イソステアリン酸', '\n', 'HbA1c', '検出抗体', '1', '\n', '原料', '3', ':', 'p-トルエンスルホン酸', ' ', '0.4', ' ', 'mol', '%', '\n', 'EPPS', '\n', 'Bis', '-', 'Tris', '\n', '原料', '1', ':', 'グリセリン', '\n', 'HEPES', '\n', 'HbA1c', '検出抗体', '2', '\n', 'CHES', '\n', '精製水', '\n', '原料', '4', ':', 'トルエン', ' ', '反応', '系', 'に', '対し', 'て', '約', '30', ' ', 'wt', '%', '\n', '原料', '

### 含む、同時に含む

In [70]:
word = 'Bis-Tris'
word = '4Tris'
def search_word(word, section, trace=False):
    n = 0
    for k in list(notes_dic.keys()):
        if word in notes_dic[k][section]:
            n += 1
            if n == 1:
                if trace:
                    print(f"\n# {word}:\n# {k}: section ====")
                    print(f"{'\n'.join([l for l in notes_dic[k][section].split('\n') if word in l])}")
    if n >= 1:
        flg = True
        if trace:
            print(f"   Other Matches: {n-1}")
    elif n < 1:
        flg = False
        if trace:
            print(f"# {word}: No Match")
    return n

def has_multiple_matches(words, section, trace=False):
    flgs = []
    for wd in words:
        if wd != '':
            flgs.append(search_word(wd, section, trace=trace))
    if sum(flgs) >= 2:
        print(f"\n### {sum(flgs)}, {flgs}, {words}")
    else:
        print(f"\n--- {sum(flgs)}, {flgs}, {words}")


In [9]:
words = ['','','','','']
words = ['HbA1c','HbA1C','ヘモグロビンA1c','グリコヘモグロビン']
#words = ['IgG','ヘペス','IgG抗体','ＩｇＧ','']
#words = ['2-EH','2-エチルヘキサノール','','','']

words_list = [
    ['HbA1c','HbA1C','ヘモグロビンA1c','グリコヘモグロビン'],
    ['NaOH','水酸化ナトリウム','水酸化Na','',''],
    ['NaCl','塩化ナトリウム','食塩','',''],
    ['Tris','トリス','Tris塩酸','Bis-Tris',''],
    ['HEPES','ヘペス','','',''],
    ['IgG','IgG抗体','ＩｇＧ','',''],
    ["f(ab')2","F(ab')2",'','',''],
    ['HRP','西洋ワサビペルオキシダーゼ','','',''],
    ['BSA','牛血清アルブミン','','',''],
    ['トリメチロールプロパン','TMP','Trimethylolpropane','',''],
    ['ペンタエリスリトール','PE','Pentaerythritol','',''],
    ['ネオペンチルグリコール','NPG','Neopentyl glycol','',''],
    ['2-エチルヘキサノール','2-EH','オクチルアルコール','',''],
]

section = 'all'
for words in words_list:
    has_multiple_matches(words, section, trace=True)


# HbA1c:
# 実験ノート ID1-1: section ====
現行HbA1c測定試薬（HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）とHbA1c検出抗体1（マウスモノクローナル，HRP標識，全長IgG）、NaOH 10mLとTris 10mL、ラテックス粒径100nmと200nm）の条件を基準条件として設定し、感度・直線性・バックグラウンドなど基本性能を初期評価する。
- ② HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）：1 mL
- ③ HbA1c検出抗体1（マウスモノクローナル，HRP標識，全長IgG）：1 mL
3-1.現行機種Xにチューブをセッティングし、HbA1cを測定・記録
   Other Matches: 289
# HbA1C: No Match
# ヘモグロビンA1c: No Match
# グリコヘモグロビン: No Match

--- 1, [True, False, False, False], ['HbA1c', 'HbA1C', 'ヘモグロビンA1c', 'グリコヘモグロビン']

# NaOH:
# 実験ノート ID1-1: section ====
現行HbA1c測定試薬（HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）とHbA1c検出抗体1（マウスモノクローナル，HRP標識，全長IgG）、NaOH 10mLとTris 10mL、ラテックス粒径100nmと200nm）の条件を基準条件として設定し、感度・直線性・バックグラウンドなど基本性能を初期評価する。
- ④ NaOH：10 mL
   Other Matches: 113
# 水酸化ナトリウム: No Match
# 水酸化Na: No Match

--- 1, [True, False, False], ['NaOH', '水酸化ナトリウム', '水酸化Na', '', '']

# NaCl:
# 実験ノート ID1-100: section ====
- ⑤ NaCl：4 mL
   Other Matches: 139
# 塩化ナトリウム: No Match
# 食塩: No Match

--- 1, [True, False, False], ['NaCl', '塩化ナトリウム', '食塩', '', '']

# T

### 隣接するトークン

In [71]:
def get_adjacent_tokens(section):
    all_adjacent_tokens = []
    for k in list(notes_dic.keys()):
        tokens = tokenize(notes_dic[k][section].translate(ZEN2HAN))
        adjacent_tokens = [a + b for a, b in zip(tokens, tokens[1:])] # 1つずらして連結

        s = pd.Series(adjacent_tokens)
        s = s.str.replace(r"^[\s,.、。:・()~+/ぁ-ゔ～]+|[\s,.、。:・()~+/ぁ-ゔ～]+$", '', regex=True) # 不要な隣接トークン
        s = s[
                ~s.str.contains(r"^[0-9,.]*$", na=False)  # 数値のトークンを除く
                & (s.str.len() > 1)                       # 長さ1以下を除外
            ]
        filtered_tokens = s[s.ne('')].tolist() # 空の要素を除く

        if filtered_tokens:
            all_adjacent_tokens.append(filtered_tokens)
    return all_adjacent_tokens

section = 'all'
adjacent_tokens = get_adjacent_tokens(section)
flat = set(
    [x
        for sub in adjacent_tokens or [] if sub is not None
            for x in sub]
)
for t in sorted(flat):
    if len(tokenize(t))<=1:
        print(f"{t}, {len(tokenize(t))}: {tokenize(t)}, {search_word(t,'all',trace=False)}")
for t in sorted(flat):
    if len(tokenize(t))>1:
        print(f"{t}, {len(tokenize(t))}: {tokenize(t)}, {search_word(t,'all',trace=False)}")


def has_multiple_matches(words, trace=False):
    flgs = []
    for wd in words:
        if wd != '':
            flgs.append(search_word(wd, trace=trace))
    if sum(flgs) >= 2:
        print(f"\n### {sum(flgs)}, {flgs}, {words}")
    else:
        print(f"\n--- {sum(flgs)}, {flgs}, {words}")

1.①, 1: ['1.①'], 200
2-EH, 1: ['2-EH'], 4
2-エチルヘキサノール, 1: ['2-エチルヘキサノール'], 6
3.④, 1: ['3.④'], 100
3.⑤, 1: ['3.⑤'], 100
Amp, 1: ['Amp'], 290
Arrhenius, 1: ['Arrhenius'], 1
BSA, 1: ['BSA'], 70
Bis, 1: ['Bis'], 8
CHES, 1: ['CHES'], 142
CLEIA, 1: ['CLEIA'], 2
CO, 1: ['CO'], 7
CV, 1: ['CV'], 29
Casein, 1: ['Casein'], 2
Dean, 1: ['Dean'], 3
EDTA, 1: ['EDTA'], 1
EPPS, 1: ['EPPS'], 7
Gelatin, 1: ['Gelatin'], 3
Glycerol, 1: ['Glycerol'], 10
Glycine, 1: ['Glycine'], 16
Good, 1: ['Good'], 3
HEPES, 1: ['HEPES'], 53
HPLC, 1: ['HPLC'], 1
HRP, 1: ['HRP'], 190
HbA1c, 1: ['HbA1c'], 290
IFCC, 1: ['IFCC'], 1
IgG, 1: ['IgG'], 290
JIS, 1: ['JIS'], 50
KCl, 1: ['KCl'], 4
KOH, 1: ['KOH'], 50
LoD, 1: ['LoD'], 13
MOPS, 1: ['MOPS'], 2
NGSP, 1: ['NGSP'], 1
NPG, 1: ['NPG'], 22
Na, 1: ['Na'], 240
NaCO, 1: ['NaCO'], 1
NaCl, 1: ['NaCl'], 140
NaHCO, 1: ['NaHCO'], 7
NaHCO3, 1: ['NaHCO3'], 7
NaN, 1: ['NaN'], 1
NaOH, 1: ['NaOH'], 114
PBS, 1: ['PBS'], 35
PE, 1: ['PE'], 68
PEG, 1: ['PEG'], 1
PVP, 1: ['PVP'], 1
Sorbitol, 1: